In [ ]:
import os

import numpy as np
import torch

from lib.dataset.dataset2D import VortexDataset2D
from lib.vortex.modules.efficientdet.efficientdet import EfficientDet 
from lib.config.project_manager import ProjectManager

project = ProjectManager()
project.load('Test_Crop')

#project.create_new(
#    name = 'Test_Crop',
#    dataset2D_path = '/home/timo/Documents/CombinedDataset',
#    dataset3D_path = None)

cfg = project.get_cfg()

In [2]:
training_set = VortexDataset2D(cfg, set='train', mode = 'cropping')
val_set = VortexDataset2D(cfg, set='val', mode = 'cropping')

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
/home/timo/Desktop/VoRTEx/datasets/Combined_Dataset/train/Colleen_03032021_Cara/Object_16/Camera_T/Frame_58043.jpg
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
/home/timo/Desktop/VoRTEx/datasets/Combined_Dataset/val/Colleen_03032021_Cara/Object_16/Camera_T/Frame_59682.jpg


In [ ]:
efficientDet = EfficientDet('train', cfg, '/home/timo/Desktop/VoRTEx/projects/Test_Crop/models/efficientdet/Run_20210705-215016/efficientdet-d0_50.pth')

In [ ]:
efficientDet.train(training_set, val_set, 100)

In [3]:
import cv2
import lib.vortex.modules.efficientdet.utils as utils
efficientDet = EfficientDet('inference', cfg, '/home/timo/Desktop/VoRTEx/projects/Test_Crop/models/efficientdet/Run_20210705-215016/efficientdet-d0_50.pth')
data = val_set.__getitem__(119)
cv2.imshow('', data[0].numpy())
cv2.waitKey(0)
regression, classification, anchors = efficientDet.model(data[0].permute(2, 0, 1).view(1,3,256,256).cuda())
regressBoxes = utils.BBoxTransform()
clipBoxes = utils.ClipBoxes()
threshold = 0.2
iou_threshold = 0.2
out = utils.postprocess(data[0].permute(2, 0, 1).view(1,3,256,256),
                  anchors, regression, classification,
                  regressBoxes, clipBoxes,
                  threshold, iou_threshold)
print (out)
out_dict = efficientDet.predict_on_image('/home/timo/Desktop/VoRTEx/datasets/Combined_Dataset/val/Colleen_03032021_Cara/Object_8/Camera_B/Frame_31274.jpg')
efficientDet.visualize_prediction(out_dict)

/home/timo/Desktop/VoRTEx/datasets/Combined_Dataset/val/Colleen_03032021_Cara/Object_8/Camera_B/Frame_31274.jpg
[{'rois': array([[119.90513 , 119.505585, 143.52608 , 169.51566 ]], dtype=float32), 'class_ids': array([0]), 'scores': array([0.99771035], dtype=float32)}]
(256, 256, 3) 2.6399999999999997
0.025632143020629883
599 521 718 770


In [ ]:

print (cfg.DATASET.DATASET_2D)